# Trabajo Practico 2 - Machine Learning

In [91]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Importamos utilidades y modelos de sklearn
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model

# Importamos los datos de entrenamiento

In [92]:
train = pd.read_csv('../Archivos CSV/TP2 -MachineLearning/train.csv'
            ,dtype={
             "id":np.int32
            ,"titulo":'category'
            ,"fecha":str 
            ,"ciudad": 'category'
            ,"provincia": 'category'
            ,"tipodepropiedad": 'category' })

# Importamos los datos de Test

Sobre este set de datos no tengo la columna 'precio' que es justamento lo que quiero predecir

In [93]:
test = pd.read_csv('../Archivos CSV/TP2 -MachineLearning/test.csv'
            ,dtype={
             "id":np.int32
            ,"titulo":'category'
            ,"fecha":str 
            ,"ciudad": 'category'
            ,"provincia": 'category'
            ,"tipodepropiedad": 'category' })

El formato del Submit es ( Id , Precio ) los ids son los que estan en el set de Test y el Precio el que predice mi modelo

 # Métrica: Root Mean Squared Logarithmic Error

In [94]:
# Métrica de evaluación
def RMSLE(actual, pred):
    return (np.mean((np.log(actual + 1) - np.log(pred + 1)) ** 2)) **.5

# 1º - Modelo: Constante

In [95]:
# Tiramos todas las columnas no numéricas para que sklearn nos acepte el dataframe. 
# Igualmente, el DummyRegressor no va a utilizar ninguna más que el precio 
drop_cols = ['fecha', 'ciudad', 'idzona', 'tipodepropiedad', 'provincia', 'titulo', 'descripcion', 'direccion']
X = train.drop(['precio'] + drop_cols, axis=1)
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
print(f"Train shapes: X={X_train.shape} y={y_train.shape}")
print(f"Test  shapes: X={X_test.shape}  y={y_test.shape}")

Train shapes: X=(180000, 14) y=(180000,)
Test  shapes: X=(60000, 14)  y=(60000,)


In [98]:
# Create a dummy regressor
dummy_constant  = DummyRegressor(strategy='constant', constant=500)

# "Train" dummy regressor ( Aca ya estoy listo para predecir)
dummy_constant.fit(X_train[['id']], y_train)

DummyRegressor(constant=array(500), quantile=None, strategy='constant')

In [99]:
#Predigo valores Puntales modo de ejemplo
dummy_constant.predict(np.array([[100]]))

array([500])

In [100]:
#Calculo de Errores contra mi set de test
pred = dummy_constant.predict(    test[['id']]     )
dummy_rmsle = RMSLE(y_test, pred)
#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, dummy_constant.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 8.23876
RMSLE Error (Test): 8.23005


# 2º Modelo Promedio

In [102]:
# Create a dummy regressor
dummy_mean = DummyRegressor(strategy='mean')

# "Train" dummy regressor
dummy_mean.fit(X_train, y_train)

DummyRegressor(constant=None, quantile=None, strategy='mean')

In [103]:
#Calculo de Errores contra mi set de test
pred = dummy_mean.predict(    test[['id']]     )
dummy_rmsle = RMSLE(y_test, pred)
#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, dummy_mean.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.90228
RMSLE Error (Test): 0.90318


In [74]:
#Exporto csv con la prediccion
pred = dummy_mean.predict(test) 
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
res = test['id'].to_frame()
res.insert(1,'target', pred , True) 
#Exportamos la prediccion en formato csv sin labels
res.to_csv(r'C:\Users\Giova\JupiterLabFolder\Archivos CSV\res.csv',index=False , header=True)

# 3º Modelo Regresion Lineal 

### metroscubiertos

In [104]:
X = train[['metroscubiertos']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
#print(f"Train shapes: X={X_train.shape} y={y_train.shape}")
#print(f"Test  shapes: X={X_test.shape}  y={y_test.shape}")

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)
#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.65673
RMSLE Error (Test): 0.65648


In [183]:
#Exporto csv con la prediccion
pred = reg.predict(test['metroscubiertos'].to_frame().fillna(X_test.mean())   ) 
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
res = test['id'].to_frame()
res.insert(1,'target', pred , True) 
#Exportamos la prediccion en formato csv sin labels
res.to_csv(r'C:\Users\Giova\JupiterLabFolder\Archivos CSV\res.csv',index=False , header=True)

### metrostotales

In [195]:
X = train[['metrostotales']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
print(f"Train shapes: X={X_train.shape} y={y_train.shape}")
print(f"Test  shapes: X={X_test.shape}  y={y_test.shape}")

Train shapes: X=(180000, 1) y=(180000,)
Test  shapes: X=(60000, 1)  y=(60000,)


In [196]:
#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

In [197]:
#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [198]:
#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)
#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.77069
RMSLE Error (Test): 0.76986


### baños 

In [207]:
X = train[['banos']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
print(f"Train shapes: X={X_train.shape} y={y_train.shape}")
print(f"Test  shapes: X={X_test.shape}  y={y_test.shape}")

Train shapes: X=(180000, 1) y=(180000,)
Test  shapes: X=(60000, 1)  y=(60000,)


In [208]:
#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

In [209]:
#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [210]:
#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)
#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.71305
RMSLE Error (Test): 0.71311


### garages

In [212]:
X = train[['garages']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.84448
RMSLE Error (Test): 0.84314


### habitaciones

In [213]:
X = train[['habitaciones']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.85682
RMSLE Error (Test): 0.85757


### gimnasio

In [214]:
X = train[['gimnasio']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.89044
RMSLE Error (Test): 0.89150


### usosmultiples

In [215]:
X = train[['usosmultiples']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.89135
RMSLE Error (Test): 0.89216


### piscina

In [216]:
X = train[['piscina']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.89413
RMSLE Error (Test): 0.89515


### longitud del titulo

In [218]:
train['tituloLen'] = train['titulo'].apply(len)
X = train[['tituloLen']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.90225
RMSLE Error (Test): 0.90318


### tipo de propiedad

In [219]:

a = train

cleanup_nums = {"tipodepropiedad":     {
    "Casa": 1,
    "Apartamento": 2, 
    "Casa en condominio": 3,
    "Terreno": 4,
    "Local Comercial": 5,
    "Oficina comercial": 6,
    "Bodega comercial": 7,
    "Edificio": 8,
    "Terreno comercial": 9,
    "Casa uso de suelo": 10,
    "Quinta Vacacional": 11,
    "Duplex": 12,
    "Villa": 13,
    "Inmuebles productivos urbanos": 14,
    "Rancho": 15,
    "Local en centro comercial": 16,
    "Departamento Compartido": 17,
    "Otros": 18,
    "Nave industrial": 19,
    "Terreno industrial": 20,
    "Huerta": 21,
    "Lote": 22,
    "Hospedaje": 23,
    "Garage": 24
                                   
}  }

a.replace(cleanup_nums, inplace=True)


X = a[['tipodepropiedad']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.90122
RMSLE Error (Test): 0.90258


### fecha : año

In [229]:
a = train
a['fecha'] = pd.to_datetime(a['fecha'])
a['anio'] = a['fecha'].dt.year

X = a[['anio']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.89211
RMSLE Error (Test): 0.89394


# 4º Modelo : Regresion Lineal Multiple

### 1

In [233]:
X = train[['metrostotales','metroscubiertos','banos']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.62025
RMSLE Error (Test): 0.61993


### 2   

In [27]:
X = train[['metrostotales','metroscubiertos','banos','habitaciones','gimnasio','usosmultiples','piscina']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.60806
RMSLE Error (Test): 0.60794


C:\Users\Giova\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Giova\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
b = test[['metrostotales','metroscubiertos','banos','habitaciones','gimnasio','usosmultiples','piscina']]
b = b.fillna(b.mean())

#Exporto csv con la prediccion
pred = reg.predict(b) 
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
res = test['id'].to_frame()
res.insert(1,'target', pred , True) 
#Exportamos la prediccion en formato csv sin labels
res.to_csv(r'C:\Users\Giova\JupiterLabFolder\Archivos CSV\res.csv',index=False , header=True)

## 3

In [7]:
X = train[['metrostotales','metroscubiertos','banos','gimnasio','habitaciones','antiguedad']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.60840
RMSLE Error (Test): 0.60735


C:\Users\Giova\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Giova\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


## 4

In [39]:
a = train
a['fecha'] = pd.to_datetime(a['fecha'])
a['anio'] = a['fecha'].dt.year
a['mes'] = a['fecha'].dt.month

X = a[['anio','mes','metrostotales','centroscomercialescercanos','metroscubiertos','banos','habitaciones','idzona','gimnasio','usosmultiples']]
y = a['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.61060
RMSLE Error (Test): 0.60723


C:\Users\Giova\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Giova\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


# 5º Modelo : Random Forest Regresion

In [105]:
# Load libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn import datasets

## 1 (Score 716812.39431)

In [6]:
# Load data with only two features

X = train[['metrostotales','centroscomercialescercanos','metroscubiertos','banos','habitaciones','idzona','gimnasio','usosmultiples']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

# Create decision tree classifer object
regr = RandomForestRegressor(random_state=0, n_jobs=-1,max_depth = 20,min_samples_split =20 ,n_estimators =50)

# Train model
model = regr.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = model.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, model.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")



RMSLE Error (train): 0.33201
RMSLE Error (Test): 0.41936


In [94]:
b = test[['metrostotales','centroscomercialescercanos','metroscubiertos','banos','habitaciones','idzona','gimnasio','usosmultiples']]
b = b.fillna(b.mean())

#Exporto csv con la prediccion
pred = model.predict(b) 
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
res = test['id'].to_frame()
res.insert(1,'target', pred , True) 
#Exportamos la prediccion en formato csv sin labels
res.to_csv(r'C:\Users\Giova\JupiterLabFolder\Archivos CSV\res.csv',index=False , header=True)

## 2   ( score 702024.74347 ) 

In [137]:
# Load data with only two features

X = train[['metrostotales','centroscomercialescercanos','metroscubiertos','banos','habitaciones','idzona','gimnasio','usosmultiples']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

# Create decision tree classifer object
regr = RandomForestRegressor(random_state=0, n_jobs=-1,max_depth = 100,max_features =6,min_samples_split = 4 ,n_estimators =200)

# Train model
model = regr.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = model.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, model.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.22532
RMSLE Error (Test): 0.40942


In [138]:
b = test[['metrostotales','centroscomercialescercanos','metroscubiertos','banos','habitaciones','idzona','gimnasio','usosmultiples']]
b = b.fillna(b.mean())

#Exporto csv con la prediccion
pred = model.predict(b) 
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
res = test['id'].to_frame()
res.insert(1,'target', pred , True) 
#Exportamos la prediccion en formato csv sin labels
res.to_csv(r'C:\Users\Giova\JupiterLabFolder\Archivos CSV\res.csv',index=False , header=True)

## 3 (Score 669492.31492) 

In [217]:
a = train
a['fecha'] = pd.to_datetime(a['fecha'])
a['anio'] = a['fecha'].dt.year
a['mes'] = a['fecha'].dt.month
a['dia'] = a['fecha'].dt.dayofweek

# Load data with only two features

X = a[['anio','mes','metrostotales','centroscomercialescercanos','piscina','escuelascercanas','metroscubiertos','banos','habitaciones','lat','lng','idzona','gimnasio','usosmultiples']]
y = a['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

# Create decision tree classifer object
regr = RandomForestRegressor(random_state=0,bootstrap =False,min_samples_leaf=2 ,n_jobs=-1,max_depth = 100,max_features = 8 ,min_samples_split = 2 ,n_estimators =99)

# Train model
model = regr.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = model.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, model.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.12171
RMSLE Error (Test): 0.39234


In [219]:
c = test
c['fecha'] = pd.to_datetime(c['fecha'])
c['anio'] = c['fecha'].dt.year
c['mes'] = c['fecha'].dt.month
c['dia'] = c['fecha'].dt.dayofweek

b = c[['anio','mes','metrostotales','centroscomercialescercanos','piscina','escuelascercanas','metroscubiertos','banos','habitaciones','lat','lng','idzona','gimnasio','usosmultiples']]
b = b.fillna(b.mean())

#Exporto csv con la prediccion
pred = model.predict(b) 
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
res = test['id'].to_frame()
res.insert(1,'target', pred , True) 
#Exportamos la prediccion en formato csv sin labels
res.to_csv(r'C:\Users\Giova\JupiterLabFolder\Archivos CSV\res.csv',index=False , header=True)

## 4  score 583658.64804

In [13]:

a = train

cleanup_nums = {"tipodepropiedad":     {
    "Casa": 1,
    "Apartamento": 2, 
    "Casa en condominio": 3,
    "Terreno": 4,
    "Local Comercial": 5,
    "Oficina comercial": 6,
    "Bodega comercial": 7,
    "Edificio": 8,
    "Terreno comercial": 9,
    "Casa uso de suelo": 10,
    "Quinta Vacacional": 11,
    "Duplex": 12,
    "Villa": 13,
    "Inmuebles productivos urbanos": 14,
    "Rancho": 15,
    "Local en centro comercial": 16,
    "Departamento Compartido": 17,
    "Otros": 18,
    "Nave industrial": 19,
    "Terreno industrial": 20,
    "Huerta": 21,
    "Lote": 22,
    "Hospedaje": 23,
    "Garage": 24
                                   
}  }

a = a.replace(cleanup_nums)
#a['tituloLen'] = a['titulo'].apply(len)

a['fecha'] = pd.to_datetime(a['fecha'])
a['anio'] = a['fecha'].dt.year
a['mes'] = a['fecha'].dt.month
a['dia'] = a['fecha'].dt.dayofweek

a = a.fillna(X_train.mean())

# Load data with only two features

X = a[['tipodepropiedad','anio','metrostotales','antiguedad','centroscomercialescercanos','garages','piscina','metroscubiertos','banos','habitaciones','lat','lng','idzona','gimnasio','usosmultiples']]
y = a['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

# Create decision tree classifer object
regr = RandomForestRegressor(random_state=0,bootstrap =False,min_samples_leaf=2 ,n_jobs=-1,max_depth = 100,max_features = 5 ,min_samples_split = 2 ,n_estimators = 400)

# Train model
model = regr.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = model.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, model.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.13892
RMSLE Error (Test): 0.35932


In [10]:
c = test
cleanup_nums = {"tipodepropiedad":     {
    "Casa": 1,
    "Apartamento": 2, 
    "Casa en condominio": 3,
    "Terreno": 4,
    "Local Comercial": 5,
    "Oficina comercial": 6,
    "Bodega comercial": 7,
    "Edificio": 8,
    "Terreno comercial": 9,
    "Casa uso de suelo": 10,
    "Quinta Vacacional": 11,
    "Duplex": 12,
    "Villa": 13,
    "Inmuebles productivos urbanos": 14,
    "Rancho": 15,
    "Local en centro comercial": 16,
    "Departamento Compartido": 17,
    "Otros": 18,
    "Nave industrial": 19,
    "Terreno industrial": 20,
    "Huerta": 21,
    "Lote": 22,
    "Hospedaje": 23,
    "Garage": 24
                                   
}  }

c = c.replace(cleanup_nums)
c['fecha'] = pd.to_datetime(c['fecha'])
c['anio'] = c['fecha'].dt.year

c = c.fillna(X_train.mean())

b = c[['tipodepropiedad','anio','metrostotales','idzona','antiguedad','centroscomercialescercanos','garages','piscina','metroscubiertos','banos','habitaciones','lat','lng','idzona','gimnasio','usosmultiples']]
b = b.fillna(b.mean())

#Exporto csv con la prediccion
pred = model.predict(b) 
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
res = test['id'].to_frame()
res.insert(1,'target', pred , True) 
#Exportamos la prediccion en formato csv sin labels
res.to_csv(r'C:\Users\Giova\JupiterLabFolder\Archivos CSV\res.csv',index=False , header=True)

C:\Users\Giova\Anaconda3\lib\site-packages\pandas\core\generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
C:\Users\Giova\Anaconda3\lib\site-packages\pandas\core\frame.py:4244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


## 5

In [88]:
#Preparo los datos

cleanup_nums = {"tipodepropiedad":     {
    "Casa": 1,
    "Apartamento": 2, 
    "Casa en condominio": 3,
    "Terreno": 4,
    "Local Comercial": 5,
    "Oficina comercial": 6,
    "Bodega comercial": 7,
    "Edificio": 8,
    "Terreno comercial": 9,
    "Casa uso de suelo": 10,
    "Quinta Vacacional": 11,
    "Duplex": 12,
    "Villa": 13,
    "Inmuebles productivos urbanos": 14,
    "Rancho": 15,
    "Local en centro comercial": 16,
    "Departamento Compartido": 17,
    "Otros": 18,
    "Nave industrial": 19,
    "Terreno industrial": 20,
    "Huerta": 21,
    "Lote": 22,
    "Hospedaje": 23,
    "Garage": 24
                                   
}  }

a = train
a = a.replace(cleanup_nums)

a['fecha'] = pd.to_datetime(a['fecha'])
a['anio'] = a['fecha'].dt.year
a['mes'] = a['fecha'].dt.month


drop_cols = [ 'ciudad', 'provincia', 'titulo', 'descripcion', 'direccion']
a = a.drop(drop_cols, axis=1)

#Relleno los nulls
a[['tipodepropiedad']] = a[['tipodepropiedad']].fillna(value=3)
a[['metroscubiertos']] = a[['metroscubiertos']].fillna(value= (a['metroscubiertos'].mean())  )
a[['antiguedad']] = a[['antiguedad']].fillna(value= (a['antiguedad'].mean()) )
a[['habitaciones']] = a[['habitaciones']].fillna(value= (a['habitaciones'].mean()) )
a[['banos']] = a[['banos']].fillna(value= (a['habitaciones'].mean()))
a[['idzona']] = a[['idzona']].fillna(value=(a['idzona'].mean()))
a[['garages']] = a[['garages']].fillna(value=(a['garages'].mean()))
a[['metrostotales']] = a[['metrostotales']].fillna(value=(a['metrostotales'].mean()))
a[['lng']] = a[['lng']].fillna(value=(a['lng'].mean()))
a[['lat']] = a[['lat']].fillna(value=(a['lat'].mean()))


# Load data with only two features

X = a[['tipodepropiedad','anio','mes','metrostotales','antiguedad','garages','piscina','metroscubiertos','banos','habitaciones','lat','lng','idzona','gimnasio','usosmultiples']]
y = a['precio']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

# Create decision tree classifer object
regr = RandomForestRegressor(random_state=0,bootstrap =False,min_samples_leaf=2 ,n_jobs=-1,max_depth = 80,max_features = 10 ,min_samples_split = 2 ,n_estimators = 500)

# Train model
model = regr.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = model.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, model.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.09347
RMSLE Error (Test): 0.35775


In [89]:

cleanup_nums = {"tipodepropiedad":     {
    "Casa": 1,
    "Apartamento": 2, 
    "Casa en condominio": 3,
    "Terreno": 4,
    "Local Comercial": 5,
    "Oficina comercial": 6,
    "Bodega comercial": 7,
    "Edificio": 8,
    "Terreno comercial": 9,
    "Casa uso de suelo": 10,
    "Quinta Vacacional": 11,
    "Duplex": 12,
    "Villa": 13,
    "Inmuebles productivos urbanos": 14,
    "Rancho": 15,
    "Local en centro comercial": 16,
    "Departamento Compartido": 17,
    "Otros": 18,
    "Nave industrial": 19,
    "Terreno industrial": 20,
    "Huerta": 21,
    "Lote": 22,
    "Hospedaje": 23,
    "Garage": 24
                                   
}  }

c = test
c = c.replace(cleanup_nums)
c['fecha'] = pd.to_datetime(c['fecha'])
c['anio'] = c['fecha'].dt.year
c['mes'] = c['fecha'].dt.month

drop_cols = [ 'ciudad', 'provincia', 'titulo', 'descripcion', 'direccion']
c = c.drop(drop_cols, axis=1)

#Relleno los nulls
c[['tipodepropiedad']] = c[['tipodepropiedad']].fillna(value=3)
c[['metroscubiertos']] = c[['metroscubiertos']].fillna(value= (c['metroscubiertos'].mean())  )
c[['antiguedad']] = c[['antiguedad']].fillna(value= (c['antiguedad'].mean()) )
c[['habitaciones']] = c[['habitaciones']].fillna(value= (c['habitaciones'].mean()) )
c[['banos']] = c[['banos']].fillna(value= (c['habitaciones'].mean()))
c[['idzona']] = c[['idzona']].fillna(value=(c['idzona'].mean()))
c[['garages']] = c[['garages']].fillna(value=(c['garages'].mean()))
c[['metrostotales']] = c[['metrostotales']].fillna(value=(c['metrostotales'].mean()))
c[['lng']] = c[['lng']].fillna(value=(c['lng'].mean()))
c[['lat']] = c[['lat']].fillna(value=(c['lat'].mean()))


b = c[['tipodepropiedad','anio','mes','metrostotales','antiguedad','garages','piscina','metroscubiertos','banos','habitaciones','lat','lng','idzona','gimnasio','usosmultiples']]


#Exporto csv con la prediccion
pred = model.predict(b) 
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
res = test['id'].to_frame()
res.insert(1,'target', pred , True) 
#Exportamos la prediccion en formato csv sin labels
res.to_csv(r'C:\Users\Giova\JupiterLabFolder\Archivos CSV\res.csv',index=False , header=True)

# 6º -Modelo : KNN